# Setup of Noteboook

The follwing code clones the github repository with course files. 
Subsequently it imports all libraries and custom modules needed for this notebook

In [1]:
!git clone https://github.com/DataHow/analytics-course-scripts.git
!pip install --upgrade scipy==1.7.3

Cloning into 'analytics-course-scripts'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 56 (delta 21), reused 49 (delta 14), pack-reused 0
Unpacking objects: 100% (56/56), done.
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
# import libaries
import pandas as pd
import numpy as np
import scipy
import importlib  
import scipy.integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EmpiricalCovariance
from scipy.stats import chi2

# import custom modules
emulator = importlib.import_module("analytics-course-scripts.scripts.modules.emulator")
plothelpers = importlib.import_module("analytics-course-scripts.scripts.modules.plothelpers")

# Exploratory Analysis via PCA

Why is it important to visualize and understand the data, not only in univariate setting, but also in multivariate. What insights can multivariate analysis provide?

## Import or Generate Dataset

In the beginning of each notebook you can make a decision whether you run the notebooks with already pre-generated runs which will be just imported or you will generate your own based on the template from `00_Process_Characterization` script.

* For dataset import set `select_dataset_source = "import"`.
* For generating data set `select_dataset_source = "generate"`.

For first time running the notebook, we recommend importing the dataset.

 

In [15]:
# Select dataset source
select_dataset_source = "import"

In [16]:
if select_dataset_source == "import":
    filename = "owu.csv"
    filepath = "/content/analytics-course-scripts/scripts/datasets/"
    owu = pd.read_csv(filepath+filename,index_col=None)
    owu.columns =  ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"]
    owu.index = pd.MultiIndex.from_product([list(range(int(len(owu)/15))),list(range(15))], names=["run","time"])
    #doe = pd.read_csv(filepath+filename.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
    doe = pd.read_csv(filepath+filename.replace(".csv","_doe.csv"),index_col=None)
    doe.columns =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"]

In [17]:
if select_dataset_source == "generate":
    # Model parameters
    mu_g_max = 0.05;
    mu_d_max = 0.025;
    K_g_Glc  = 1;
    K_I_Lac  = 30;
    K_d_Lac  = 50;
    k_Glc    = 0.04;
    k_Lac    = 0.06;
    k_Prod   = 1;

    # Process parameters
    feed_start = [1, 4]
    feed_end = [8, 12]
    Glc_feed_rate = [5, 20]
    Glc_0 = [10, 80.0]
    VCD_0 = [0.1, 1.0]

    var_lims = {"mu_g_max":mu_g_max,"mu_d_max": mu_d_max,"K_g_Glc" : K_g_Glc,"K_I_Lac" : K_I_Lac,"K_d_Lac" : K_d_Lac,"k_Glc" : k_Glc,"k_Lac" : k_Lac,"k_Prod" : k_Prod,"feed_start" : feed_start, "feed_end" : feed_end,"Glc_feed_rate" : Glc_feed_rate, "Glc_0" : Glc_0, "VCD_0" : VCD_0}

    ## Define the number of experiments
    num_runs = 50
    ## Define the filename for the dataset
    filename = "owu.csv"
    filepath = "/content/"
    # Generate DOE Dataset
    owu = emulator.generate_data(var_lims, num_runs, filename)
    doe = pd.read_csv(filepath+filename.replace(".csv","_doe.csv"),index_col=None, usecols =["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"])
    owu = pd.read_csv("/content/"+filename,index_col=0)
    owu.columns =  ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"]
    owu.index = pd.MultiIndex.from_product([list(range(num_runs)),list(range(15))], names=["run","time"])

# PCA of OWU matrix

In this section, we use generated experiments using a Latin Hypercube design (LHD) from previous notebook. This will be used to create an observation-wise unfolder (OWU) matrix that will be analyzed using Principle Component Analysis (PCA).

For further information about the process emulator for cell culture fed-batch processes, check the script "00_Process_Characterization.ipynb" or the Powerpoint presentation "Simplified InSilico Model.pptx".

Details of OWU unfolding

### Visualize the OWU matrix

In the OWU matrix, the 1st column corresponds to VCD, the 2nd to glucose, the 3rd to lactate, the 4th to titer, and the 5th to the feed rate.


In [18]:
owu

X:VCD   X:Glc     X:Lac     X:Titer  W:Feed
run time                                                
0   0      0.65336  24.989    0.0000     0.00000   0.000
    1      1.99900  23.828    1.7454     0.12965   0.000
    2      5.31610  20.523    6.7136     2.26180  19.803
    3     10.93400  32.668   18.2210    21.07500  19.803
    4     16.31000  39.240   38.0940   100.25000  19.803
...            ...     ...       ...         ...     ...
99  10     7.07870  39.417  159.2100  1207.80000   0.000
    11     5.30480  33.518  168.0700  1303.90000   0.000
    12     3.94070  29.116  174.6800  1373.40000   0.000
    13     2.90660  25.859  179.5800  1422.30000   0.000
    14     2.13220  23.464  183.1800  1455.30000   0.000

[1500 rows x 5 columns]

In [19]:
doe

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0
0,34.416,4,12,12.4900,0.76765
1,64.099,3,9,6.1978,0.89835
2,15.436,2,9,7.4287,0.35186
3,15.777,3,9,12.3150,0.54128
4,34.904,2,9,7.2966,0.48320
...,...,...,...,...,...
94,29.443,3,9,10.6600,0.16841
95,23.982,1,11,12.0660,0.61780
96,57.927,3,11,7.9015,0.63589
97,11.654,2,12,12.6260,0.11877


### Plot correlation matrix

The OWU matrix is used to plot the degree of correlation between the different variables.

In [20]:
fig = px.imshow(owu.corr())
fig.update_layout(title='Correlation Matrix among X variables')
fig.show()

### Unnormalized PCA

PCA is run on the OWU matrix, without any variable normalization.



In [7]:
# Select number of components
select_n_components = 5

In [8]:
# Run PCA
pca = PCA(n_components = select_n_components)
pca.fit(owu)
expl_var = pca.explained_variance_
expl_var_ratio = pca.explained_variance_ratio_
pca_n_comp = list(range(1,pca.n_components_+1))

In [9]:
# Explained Variance Plot
fig = px.line(x=pca_n_comp, y=1-expl_var_ratio, color=px.Constant("Cumulative explained variance"), labels=dict(x="Principal component index", y="Explained Variance Ratio", color="Legend"))
fig.add_bar(x=pca_n_comp, y=expl_var_ratio, name="Individual explained variance")
fig.show()

### Normalized PCA

PCA is run on the OWU matrix, but this time the variables are first normalized with respect to their mean and standard deviation.


In [10]:
# Scale data by mean and standard deviation
scaler = StandardScaler()
scaled_data = scaler.fit_transform(owu)

In [11]:
# Run PCA on scaled data
pca.fit(scaled_data)
expl_var = pca.explained_variance_
expl_var_ratio = pca.explained_variance_ratio_
pca_n_comp = list(range(1,pca.n_components_+1))
components = pca.fit_transform(scaled_data)

In [12]:
# Plot Explained Variance plots
fig = px.line(x=pca_n_comp, y=1-expl_var_ratio, color=px.Constant("Cumulative explained variance"), labels=dict(x="Principal component index", y="Explained Variance Ratio", color="Legend"))
fig.add_bar(x=pca_n_comp, y=expl_var_ratio, name="Individual explained variance")
fig.show()

### Plot scores and loadings

In the following plot, the PCA loadings are plotted together with the observation scores.

Color according to run number / time step / display loadings


In [60]:
# Principal component on x-axis
select_x_pca = 1
# Principal component on y-axis
select_y_pca = 3
# Color plots by
select_color = "Run_id"
# Highlight run
select_highlight = 1

In [61]:
# Score plot of PCA
if select_color == "Run_id":
    color_idx=owu.index.get_level_values("run")
elif select_color == "Time_id":
    color_idx=owu.index.get_level_values("time")
elif select_color == "Titer_14":
    color_idx =np.repeat(np.array(owu["X:Titer"][:,14]),15)
elif select_color in doe.columns:
    color_idx = np.repeat(np.array(doe[select_color]),15)
else: 
    color_idx = None
fig = px.scatter(x=components[:,select_x_pca-1], y=components[:,select_y_pca-1], color=color_idx, title="PCA Score plot", labels={'x':"Principal Component - "+str(select_x_pca), 'y':"Principal Component - "+str(select_y_pca), 'color':str(select_color)}, )    
fig.show()

In [62]:
# Loading plot of PCA
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
features = list(owu.columns)

fig = px.scatter(x=[0,0], y=[0,0], labels={'x':"Principal Component - "+str(select_x_pca), 'y':"Principal Component - "+str(select_y_pca)}, title="PCA Loading plot")
for i, feature in enumerate(features):
    fig.add_shape(type='line', x0=0, y0=0, x1=loadings[i, 0], y1=loadings[i, 1])
    fig.add_annotation(x=loadings[i, 0], y=loadings[i, 1], ax=0, ay=0, xanchor="center", yanchor="bottom", text=feature)
fig.show()


In [51]:
# Mahalanobis distance of observations (equal to the euclidean distance in PCA space)
emp_cov = EmpiricalCovariance().fit(scaled_data)
mahalanobis = emp_cov.mahalanobis(scaled_data)
T2threshold = chi2.ppf(0.95, scaled_data.shape[1]-1)
fig = px.bar(x=list(range(len(mahalanobis))),y=mahalanobis, labels=dict(x="Observation Id.", y="Mahalanobis distance"), title="Mahalanobis distance with 95% Confidence Interval")
fig.add_hline(y=T2threshold)
fig.show()


### Task: Normalize by median VCD and run PCA on OWU matrix






# PCA of BWU matrix

Details of BWU unfolding

### Visualize the BWU matrix

In the BWU matrix, the 1st column corresponds to VCD at day 0, the 2nd to glucose at day 0, the 3rd to lactate at day 0, the 4th to titer at day 0, and the 5th to the feed rate at day 0. This is repeated for day 1 and the following days. So, column 6 corresponds to VCD at day1, column 7 corresponds to Glc at day1, ..., column 11 corresponds to VCD at day2, etc.

In [14]:
# Transform OWU to BWY
for run_ix,run in owu.groupby("run"):
    if run_ix == 0: # fix
        bwum = run.unstack(level=1)
    else:
        bwum = pd.concat([bwum, run.unstack(level=1)])

In [15]:
#bwu_columns = [' '.join(col).strip() for col in bwum.columns.values]
bwu_columns = [str(bwum.columns.get_level_values(0)[i])+str(":")+str(bwum.columns.get_level_values(1)[i]) for i in range(len(bwum.columns.get_level_values(0)))]
bwu = pd.DataFrame(bwum.to_numpy(), columns=bwu_columns)
bwu

,X:VCD:0,X:VCD:1,X:VCD:2,X:VCD:3,X:VCD:4,X:VCD:5,X:VCD:6,X:VCD:7,X:VCD:8,X:VCD:9,...,W:Feed:5,W:Feed:6,W:Feed:7,W:Feed:8,W:Feed:9,W:Feed:10,W:Feed:11,W:Feed:12,W:Feed:13,W:Feed:14
0,0.65336,1.99900,5.3161,10.9340,16.3100,18.572,17.726,15.155,11.9800,8.7291,...,19.8030,19.8030,19.8030,19.8030,19.8030,19.8030,19.803,0.0,0.0,0.0
1,0.76765,2.36320,6.1927,12.0990,16.7580,17.359,14.094,10.769,8.5544,6.7979,...,12.4900,12.4900,12.4900,12.4900,12.4900,12.4900,12.490,0.0,0.0,0.0
2,0.89835,2.78170,7.1142,13.2490,17.8130,18.933,17.368,14.487,11.1130,7.4140,...,6.1978,6.1978,6.1978,6.1978,0.0000,0.0000,0.000,0.0,0.0,0.0
3,0.35186,1.06500,2.9702,6.9794,12.4310,16.314,15.611,11.045,7.8400,5.8974,...,7.4287,7.4287,7.4287,7.4287,0.0000,0.0000,0.000,0.0,0.0,0.0
4,0.54128,1.62400,4.3240,8.9543,13.6260,16.084,15.510,12.879,10.2240,8.1402,...,12.3150,12.3150,12.3150,12.3150,0.0000,0.0000,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.16841,0.53261,1.6148,4.3714,9.4284,14.931,17.911,17.771,15.6530,12.7900,...,10.6600,10.6600,10.6600,10.6600,0.0000,0.0000,0.000,0.0,0.0,0.0
96,0.61780,1.89020,5.1061,10.7140,16.1920,18.642,17.996,15.616,12.6980,9.9173,...,12.0660,12.0660,12.0660,12.0660,12.0660,12.0660,0.000,0.0,0.0,0.0
97,0.63589,1.99550,5.4246,11.1930,16.5490,18.754,17.921,15.408,12.3620,9.4525,...,7.9015,7.9015,7.9015,7.9015,7.9015,7.9015,0.000,0.0,0.0,0.0
98,0.11877,0.35630,1.0376,2.8817,6.9104,12.560,16.885,18.088,16.7760,14.2470,...,12.6260,12.6260,12.6260,12.6260,12.6260,12.6260,12.626,0.0,0.0,0.0


### Plot correlation matrix
The BWU matrix is used to plot the degree of correlation between the different variables.


In [16]:
fig = px.imshow(bwu.corr())
fig.update_layout(title='BWU Correlation Matrix among X variables')
fig.show()

### Normalized PCA

PCA is run on the BWU matrix, bu this time the variables are first normalized with respect to their mean and standard deviation.


In [17]:
# Select number of components
select_n_components = 10

In [18]:
pca = PCA(n_components=min(min(bwu.shape),select_n_components))
# Scale data by mean and standard deviation
scaler = StandardScaler()
scaled_data = scaler.fit_transform(bwu)
pca.fit(scaled_data)
expl_var = pca.explained_variance_
expl_var_ratio = pca.explained_variance_ratio_
pca_n_comp = list(range(1,pca.n_components_+1))
components = pca.fit_transform(scaled_data)

In [22]:
# Plot Explained Variance plots
fig = px.line(x=pca_n_comp, y=1-expl_var_ratio, color=px.Constant("Cumulative explained variance"), labels=dict(x="Principal component index", y="Explained Variance Ratio", color=""))
fig.add_bar(x=pca_n_comp, y=expl_var_ratio, name="Individual explained variance")
fig.show()

### Plot scores and loadings

In the following plot, the PCA loadings are plotted first with the loadings and then without.


In [23]:
# Principal component on x-axis
select_x_pca = 1
# Principal component on y-axis
select_y_pca = 2
# Color plots by
select_coloring = None

In [24]:
# Score plot of PCA
fig = px.scatter(x=components[:,select_x_pca-1], y=components[:,select_y_pca-1], labels={'x':"Principal Component - "+str(select_x_pca), 'y':"Principal Component - "+str(select_y_pca)}, title="PCA BWU Score plot")
fig.show()

In [25]:
# Loading plot of PCA
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
features = list(bwu.columns)

fig = px.scatter(x=[0,0], y=[0,0], labels={'x':"Principal Component - "+str(select_x_pca), 'y':"Principal Component - "+str(select_y_pca)}, title="PCA Loading plot")
for i, feature in enumerate(features):
    fig.add_shape(type='line', x0=0, y0=0, x1=loadings[i, 0], y1=loadings[i, 1])
    fig.add_annotation(x=loadings[i, 0], y=loadings[i, 1], ax=0, ay=0, xanchor="center", yanchor="bottom", text=feature)
fig.show()


In [26]:
# Select which loading to plot
select_x_pca = 1

In [27]:
# Loading plot of PCA alternative
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading = loadings[:,select_x_pca-1]
features = list(bwu.columns)

fig = px.bar(x=features, y=loading,labels={'x':"Variable", 'y':"Loading of PC - "+str(select_x_pca)}, title="PCA Loading plot")
fig.show()

### Task: Compute Mahalanobis distances on BWU matrix

# Solutions to tasks


# Main focus process characterization
* degrees of freedom
* univariate visualization
* abnormalities

# Main focus PCA
* abnormalities in runs/variables
* interpretation scores/loadings
* differences between OWU/BWU
